In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from  sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor, Pool
import lightgbm as lgb
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, mean_absolute_percentage_error, mean_squared_error, r2_score, mean_absolute_error
#from  typing import
import warnings
warnings.filterwarnings('ignore')
from scipy import sparse
from lightgbm import early_stopping, log_evaluation

In [ ]:
dX=pd.read_csv("data/cleaned_data_after_imputation.csv")
# Ensure X is numeric, clean, and has no missing values
X_clean = X.copy()
X_clean = X.dropna()

# (Optional) Standardize features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_clean)

# Apply PCA
pca = PCA()
pca.fit(X_scaled)

# Get the amount of variance explained by each principal component
explained_variance = pca.explained_variance_ratio_

# Create a DataFrame for feature contributions to each principal component
loadings = pd.DataFrame(
    pca.components_.T, 
    columns=[f'PC{i+1}' for i in range(len(pca.components_))],
    index=X_clean.columns
)

# Sum of absolute loadings to rank original features by their overall impact
feature_importance = loadings.abs().sum(axis=1).sort_values(ascending=False)

# Display top 20 influential features based on PCA loadings
print(feature_importance.head(20))

# Plot top N features
top_n = 50
plt.figure(figsize=(12, 8))
sns.barplot(x=feature_importance.head(top_n).values, y=feature_importance.head(top_n).index, palette="rocket")
plt.title(f"Top {top_n} Feature Importances from PCA Loadings")
plt.xlabel("Sum of Absolute Loadings")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()

NameError: name 'X' is not defined

In [ ]:
pca = PCA(n_components=80)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)
xgb_regressor = xgb.XGBRegressor(
    objective='reg:squarederror', # For regression tasks, specify the objective
    n_estimators=100,             # Number of boosting rounds
    learning_rate=0.1,            # Step size shrinkage
    max_depth=5,                  # Maximum depth of a tree
    subsample=0.8,                # Subsample ratio of the training instance
    colsample_bytree=0.8,         # Subsample ratio of columns
    random_state=42,              # For reproducibility
    n_jobs=-1                     # Use all available CPU cores
)

# Fit the model to the training data
xgb_regressor.fit(X_train_pca, y_train)

# Make predictions on the test data
xgb_predictions = xgb_regressor.predict(X_test_pca)

# Evaluate the XGBoost model
xgb_mse = mean_squared_error(y_test, xgb_predictions)
xgb_r2 = r2_score(y_test, xgb_predictions)

print(f"XGBoost MSE: {xgb_mse}")
predictions = xgb_regressor.predict(X_test_pca)
accu = r2_score(y_test, predictions)
print("Accuracy of test:", accu)

pred = xgb_regressor.predict(X_train_pca)
accu = r2_score(y_train, pred)
print("Accuracy of train:", accu)